In [13]:
import pandas as pd
import numpy as np
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import pandas as pd
import numpy as np
import itertools
import mediapipe as mp
from collections import deque

In [2]:
data = pd.read_csv('hand_landmarks_dataset.csv')

In [3]:
X, y = data.drop(columns=['label','label_id', 'path']), data['label_id']

In [4]:
def super_frame(data, sequence_length = 37, num_features = 63):
    raw_data = data.to_numpy()
    n_sequences = len(raw_data) // sequence_length
    raw_data = raw_data[:n_sequences * sequence_length]
    X_seq = raw_data.reshape((n_sequences, sequence_length, num_features))

    # "Суперкадр" — среднее по последовательности
    X_mean = np.mean(X_seq, axis=1, keepdims=True)  # (n_sequences, 1, 63)

    # Добавляем к последовательности
    X_augmented = np.concatenate([X_seq, X_mean], axis=1)  # (n_sequences, 38, 63)

    X = pd.DataFrame(X_augmented.reshape(38*n_sequences, 63))
    
    return X


In [5]:
prep_X = super_frame(X)

In [6]:
minmax = MinMaxScaler()
X_scaled = minmax.fit_transform(prep_X)

window_size_x = 38
window_size_y = 37
n_samples = len(X_scaled) // window_size_x
n_samples_y = len(y) // window_size_y

y_seq = np.array(y[:n_samples_y * window_size_y]).reshape(n_samples_y, window_size_y)

y_seq = y_seq[:, 0] 

X_scaled = X_scaled.reshape(n_samples, 38, 63)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_seq, train_size=0.8, random_state=13)

In [7]:
y_train = to_categorical(y_train, num_classes=27)
y_test = to_categorical(y_test, num_classes=27)

In [8]:
y.value_counts()

label_id
0     38184
15    16650
20    16502
16    16243
11    16169
3     16169
26    16095
18    16095
10    15873
8     15873
23    15799
24    15725
6     15725
5     15614
25    15577
19    15577
4     15466
12    15355
9     15318
1     15244
14    14948
2     14763
17    14689
13    14615
7     13986
21    11026
22    10989
Name: count, dtype: int64

In [9]:
X_train

array([[[0.08757717, 0.03034138, 0.48643002, ..., 0.14308749,
         0.25328326, 0.80484862],
        [0.08757717, 0.03034138, 0.48643002, ..., 0.14308749,
         0.25328326, 0.80484862],
        [0.08757717, 0.03034138, 0.48643002, ..., 0.14308749,
         0.25328326, 0.80484862],
        ...,
        [0.08757717, 0.03034138, 0.48643002, ..., 0.14308749,
         0.25328326, 0.80484862],
        [0.08757717, 0.03034138, 0.48643002, ..., 0.14308749,
         0.25328326, 0.80484862],
        [0.08757717, 0.03034138, 0.48643002, ..., 0.14308749,
         0.25328326, 0.80484862]],

       [[0.52077936, 0.47730463, 0.52501584, ..., 0.559212  ,
         0.48411334, 0.79895805],
        [0.52071049, 0.4760113 , 0.52500516, ..., 0.5590961 ,
         0.47315966, 0.79582027],
        [0.08757717, 0.03034138, 0.48643002, ..., 0.14308749,
         0.25328326, 0.80484862],
        ...,
        [0.55241313, 0.50270637, 0.53885878, ..., 0.44449754,
         0.46413642, 0.74665566],
        [0.5

In [18]:
best_acc = 0
best_model = 0

# Генерация всех комбинац

# Перебор комбинаций

model = Sequential([
    Bidirectional(GRU(64, return_sequences=True, input_shape=(37, 63))),
    LayerNormalization(),
    GRU(64),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dense(27, activation='softmax')
])


model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=[
        'accuracy',
        Precision(name='precision'),
        Recall(name='recall'),
        AUC(name='roc_auc', curve='ROC'),
        AUC(name='pr_auc', curve='PR')  # <-- добавь PR AUC сюда
    ]
)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)

callbacks = [early_stop]

history = model.fit(
    X_train, y_train,
    epochs=300,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

results = model.evaluate(X_test, y_test, verbose=2)
print("Loss:", results[0])
print("Accuracy:", results[1])
print("Precision:", results[2])
print("Recall:", results[3])
print("ROC AUC:", results[4])
print("PR AUC:", results[5])

model.save("best_model.h5")



Epoch 1/300
294/294 [==============================] - 8s 16ms/step - loss: 3.2703 - accuracy: 0.0800 - precision: 0.0000e+00 - recall: 0.0000e+00 - roc_auc: 0.5537 - pr_auc: 0.0476 - val_loss: 3.2372 - val_accuracy: 0.0865 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc_auc: 0.6069 - val_pr_auc: 0.0593
Epoch 2/300
294/294 [==============================] - 4s 13ms/step - loss: 3.1762 - accuracy: 0.0848 - precision: 0.0000e+00 - recall: 0.0000e+00 - roc_auc: 0.6380 - pr_auc: 0.0617 - val_loss: 3.0996 - val_accuracy: 0.0882 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc_auc: 0.6774 - val_pr_auc: 0.0707
Epoch 3/300
294/294 [==============================] - 4s 13ms/step - loss: 3.0152 - accuracy: 0.1077 - precision: 0.0000e+00 - recall: 0.0000e+00 - roc_auc: 0.7140 - pr_auc: 0.0852 - val_loss: 2.8031 - val_accuracy: 0.1627 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc_auc: 0.7798 - val_pr_auc: 0.1224
Epoch 4/300
294/294 [===================

In [ ]:
best_acc = 0
best_model = 0

# Генерация всех комбинац

# Перебор комбинаций

model = Sequential([
    Bidirectional(GRU(64, return_sequences=True, input_shape=(37, 63))),
    LayerNormalization(),
    GRU(64),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dense(27, activation='softmax')
])


model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=[
        'accuracy',
        Precision(name='precision'),
        Recall(name='recall'),
        AUC(name='roc_auc', curve='ROC'),
        AUC(name='pr_auc', curve='PR')  # <-- добавь PR AUC сюда
    ]
)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)

callbacks = [early_stop]

history = model.fit(
    X_train, y_train,
    epochs=300,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

results = model.evaluate(X_test, y_test, verbose=2)
print("Loss:", results[0])
print("Accuracy:", results[1])
print("Precision:", results[2])
print("Recall:", results[3])
print("ROC AUC:", results[4])
print("PR AUC:", results[5])


